In [ ]:
# https://adamj.eu/tech/2019/03/11/pip-install-from-a-git-repository/
# Install customized linc-detector faster-rcnn
!pip install --upgrade --force-reinstall git+https://github.com/linc-lion/LINC-detector.git@ba36a5bfa5ba7b9035977c02b1d8ed253f074e8d

In [ ]:
# Install pycocotools
!pip install --upgrade --force-reinstall cython
!pip install --upgrade --force-reinstall -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [7]:
import datetime
import os
import time
import shutil
import subprocess
import sys

import torch
import torch.utils.data
from linc.detector.models import detection
from torch.utils.tensorboard import SummaryWriter

from linc.detector.helper.coco_utils import get_coco  # get_coco_kp
from linc.detector.helper.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
from linc.detector.helper.engine import train_one_epoch, evaluate
from linc.detector.helper import utils

%load_ext tensorboard

### Model Training
* The base training code is extracted from [pytorch example](https://github.com/pytorch/vision/blob/528651a031a08f9f97cc75bd619a326387708219/references/detection/train.py)

In [8]:
# Path to COCO formatted object detection dataset
data_path = '../datasets/coco_all_but_ws_and_fb/'  

# Ignorable arguments
epochs = 2 # 35
save_every_num_epochs = None  # Optional
evaluate_every_num_epochs = 2
lr = 0.01
momentum = 0.9
weight_decay = 1e-4
lr_steps = [10, 11]
lr_gamma = 0.1
batch_size = 3
workers = 8
run_name = "linc-detector-tensorboard"  # Optional, str used to name Tensorboard summaries
num_draw_predictions = 5
draw_threshold = 0.5
aspect_ratio_group_factor = 0

In [9]:
# Training code is based on 

print("Create summary writer for Tensorboard")
if run_name:
    log_dir_path = f"{run_name}" if run_name else None
    if os.path.isdir(log_dir_path):
        delete = input(f"Summary folder '{log_dir_path}' already exists. Overwrite it [yes, y / no, n]?")
        if delete in ('yes', 'y'):
            shutil.rmtree(log_dir_path)
        else:
            print(f"Chose another run name or delete the folder then!")
            exit()
else:
    log_dir_path = None
writer = SummaryWriter(log_dir=log_dir_path)

# Add some useful text summaries (Tensorboard uses markdown to render text).
# writer.add_text('Command executed', f"python {' '.join(sys.argv)}")
# writer.add_text('Arguments', str(args).replace(", ", ",  \n").replace("Namespace(", "").replace(")", ""))

print("Create datasets")
dataset, num_classes, label_names = get_coco(data_path, image_set='train')
print(f"Categorizing into {num_classes} classes")
dataset_test, _, _ = get_coco(data_path, image_set='val')

print("Create samplers")
train_sampler = torch.utils.data.RandomSampler(dataset)
test_sampler = torch.utils.data.SequentialSampler(dataset_test)
group_ids = create_aspect_ratio_groups(dataset, k=aspect_ratio_group_factor)
train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)

print("Create dataloaders")
data_loader = torch.utils.data.DataLoader(dataset, 
                                          batch_sampler=train_batch_sampler, 
                                          num_workers=workers, 
                                          collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(dataset_test, 
                                               batch_size=1,
                                               sampler=test_sampler, 
                                               num_workers=workers, 
                                               collate_fn=utils.collate_fn)

print("Create model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
model = detection.__dict__['fasterrcnn_resnet50_fpn'](num_classes=num_classes, pretrained=False)
model.to(device)
model_without_ddp = model

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, 
                            lr=lr, 
                            momentum=momentum, 
                            weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                    milestones=lr_steps, 
                                                    gamma=lr_gamma)

print("Start training")
start_time = time.time()
for epoch in range(epochs):
    start_epoch = time.time()
    train_one_epoch(
        model, optimizer, data_loader, device, epoch, 20, writer, label_names
    )
    print(f"Epoch time {time.time() - start_epoch}")
    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step=epoch)
    lr_scheduler.step()

    if save_every_num_epochs and epoch % save_every_num_epochs == 0:
        utils.save_on_master({
            'model': model_without_ddp.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'args': args,
            'label_names': label_names},
            os.path.join(writer.log_dir, 'model_{}.pth'.format(epoch))
        )

    if epoch % evaluate_every_num_epochs == 0:
        evaluate(
            model, data_loader_test, epoch, writer, draw_threshold,
            label_names, num_draw_predictions, device=device
        )

# Save after training is done
utils.save_on_master({
    'model': model_without_ddp.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_scheduler': lr_scheduler.state_dict(),
    'label_names': label_names},
    os.path.join(writer.log_dir, 'model_finished.pth')
)

writer.close()
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


Create summary writer for Tensorboard


Summary folder 'linc-detector-tensorboard' already exists. Overwrite it [yes, y / no, n]? yes


2022-09-11 12:27:06.642054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create datasets
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Categorizing into 32 classes
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Create samplers
Using [0, 1.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [15  1]
Create dataloaders
Create model
Device: cpu
Start training
Epoch: [0]  [0/5]  eta: 0:03:52  lr: 0.002507  loss: 4.2483 (4.2483)  loss_classifier: 3.4470 (3.4470)  loss_box_reg: 0.0203 (0.0203)  loss_objectness: 0.6902 (0.6902)  loss_rpn_box_reg: 0.0908 (0.0908)  time: 46.4494  data: 3.5685  max mem: 0
Epoch: [0]  [4/5]  eta: 0:00:40  lr: 0.010000  loss: 4.1892 (4.1375)  loss_classifier: 3.4133 (3.3396)  loss_box_reg: 0.0203 (0.0224)  loss_objectness: 0.6896 (0.6885)  loss_rpn_box_reg: 0.0908 (0.0870)  time: 40.4812  data: 0.7419  max mem: 0
Epoch: [0] Total time: 0:03:22 (40.5414 s / it)
Epoch time 202.76696014404297
Test:  [0/5]  eta: 0:00:36  model_time: 5.4164 (5.4164

In [11]:
%tensorboard --logdir linc-detector-tensorboard

Reusing TensorBoard on port 6006 (pid 12351), started 0:05:51 ago. (Use '!kill 12351' to kill it.)